In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
from pyshacl import validate
import glob
import os
import regex as re
import json
import requests
import datetime

### Crawl Webpages and Scrape JSON-LD Information

In [2]:
# list of sites to scrape
# for dev, use local files

class JsonSpider(scrapy.Spider):
    
    """ 
    scrape .json-ld data from court websites.
    prefer linked .json, 
    scrape embedded data otherwise.
    return msg if no data found.
    """

    name = "court-data-spider"

    def start_requests(self):
        dev_sites = glob.glob("sites/*/*.html")  
        crawl_sites = dev_sites
        
        # GET request, pass res to parse()
        for url in crawl_sites:
            yield scrapy.Request(url=f"http://localhost:8000/{url}", callback=self.parse)

            
    def parse(self, response):

        # look for json data. 
        linked_json = response.selector.xpath(
            '//link[@type="application/ld+json"]/@href').get()
        embedded_json = response.selector.xpath(
            '//script[@type="application/ld+json"]/text()').get()

        # use page source as filename, replace "/" 
        # need a better convention
        page_source = response.url.replace("/", ".")
        filename = (page_source + ".json").replace(".html","")

        # follow link to json file and grab data
        if linked_json is not None:
            req = requests.get(linked_json)

            # append source and date metadata
            load_json = json.loads(req.content)
            load_json.append(
                {"source": filename, "accessed": str(datetime.datetime.now())})
            json_out = json.dumps(load_json)
            with open(f"raw_json/{filename}", "w") as output:
                output.write(json_out)

        # remove whitespace that is not in a value
        elif embedded_json is not None:
            embedded_json = re.sub(r'\s+[^\:\S\"]', "", embedded_json)
            load_json = json.loads(embedded_json[1:-1])

            # append source and date metadata
            load_json.append(
                {"source": response.url, "accessed": str(datetime.datetime.now())})
            json_out = json.dumps(load_json)
            with open(f"raw_json/{filename}", "w") as output:
                output.write(json_out)

process = CrawlerProcess(
    settings = {
        "DOWNLOAD_DELAY": 1,
        "CONCURRENT_REQUESTS_PER_DOMAIN": 10
    }
)

process.crawl(JsonSpider)
process.start()

2022-10-06 17:32:58 [scrapy.utils.log] INFO: Scrapy 2.6.3 started (bot: scrapybot)
2022-10-06 17:32:58 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.4, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.10.2 (v3.10.2:a58ebcc701, Jan 13 2022, 14:50:16) [Clang 13.0.0 (clang-1300.0.29.30)], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform macOS-12.5.1-x86_64-i386-64bit
2022-10-06 17:32:58 [scrapy.crawler] INFO: Overridden settings:
{'CONCURRENT_REQUESTS_PER_DOMAIN': 10, 'DOWNLOAD_DELAY': 1}
2022-10-06 17:32:58 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2022-10-06 17:32:58 [scrapy.extensions.telnet] INFO: Telnet Password: 0a96ac6b4719d9e8
2022-10-06 17:32:58 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2022-10-06 17:32:58 [

### Validate JSON-LD Files

In [3]:
# scraped JSON-LD files 
validation_files = glob.glob("./raw_json/*.json")

# SHACL file to validate data against
sg = './court-data-standard-shacl.ttl'

errors = []


for file in validation_files:
    try:
        r = validate(file,
                     shacl_graph=sg,
                     inference='none',
                     abort_on_first=True,
                     allow_infos=False,
                     allow_warnings=False,
                     meta_shacl=False,
                     advanced=True,
                     js=False,
                     debug=False)
        if r[0] != True:
            msg = r[2]
            errors.append(file + "\n" + msg)
            
    except JSONDecodeError:
        print(file + "\nBad JSON format.")
        pass


# print file names and constraint violations 
if errors: 
    for error in errors:
        print(error)

2022-10-06 17:33:16 [py.warnings] WARNING: /Users/home/.local/share/virtualenvs/court-data-pipeline-DL0ZJ-Cx/lib/python3.10/site-packages/pyshacl/constraints/core/string_based_constraints.py:279: FutureWarning: Possible nested set at position 64
  re_matcher = re.compile(re_pattern, re_flags)



./raw_json/http:..localhost:8000.sites.embedded.nm-bernalillo.json
Validation Report
Conforms: False
Results (2):
Constraint Violation in ClassConstraintComponent (http://www.w3.org/ns/shacl#ClassConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:class schema:AdministrativeArea ; sh:description Literal("The AdministrativeArea(s) contained in the AdministrativeArea being described.") ; sh:label Literal("composing administrative area") ; sh:minLength Literal("1", datatype=xsd:integer) ; sh:path schema:geoContains ]
	Focus Node: <http://schema.org/AdministrativeArea>
	Value Node: Literal("https://www.illinoiscourts.gov/Circuit/1")
	Result Path: schema:geoContains
	Message: Value does not have class schema:AdministrativeArea
Constraint Violation in ClassConstraintComponent (http://www.w3.org/ns/shacl#ClassConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:class schema:AdministrativeArea ; sh:description Literal("Represents the geographic area served by a Court